### Name: Shubham Shetty
### UID: 2018130050
### Batch: B

## Objective:
Feature Engineering: Dimensional Reduction

**Linear Discriminant Analysis (LDA)** is a technique of model distribution of predictors in each of the response classes and use Bayes Theorem to flip around into estimates for classwise probability. This approach assumes the predictors to have come from multivariate Gaussian Distribution with class specific mean vector and common covariance matrix."

## Importing Libraries and Dataset

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/Data Science Tutorials/WA_Fn-UseC_-HR-Employee-Attrition.csv')
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [5]:
data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

### Few of the predictors are useless carrying same value for all the observations ,thus having no significance in the desired output variable:
####    'EmployeeCount' , 'EmployeeNumber' , 'Over18' , 'StandardHours

In [6]:
data = data.drop(['EmployeeCount','EmployeeNumber','Over18','StandardHours'],axis=1)
data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

### Getting unique elements for every predictor variable

In [7]:
data['MaritalStatus'].unique()

array(['Single', 'Married', 'Divorced'], dtype=object)

## Replacing our Attrition output by integer constants

In [8]:
data.loc[data['Attrition']=='No','Attrition'] = 0
data.loc[data['Attrition']=='Yes','Attrition'] = 1
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,Female,...,3,1,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,Male,...,4,4,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,4,Male,...,3,2,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,Female,...,3,3,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,Male,...,3,4,1,6,3,3,2,2,2,2


## Categorising on the basis of travel for business purposes; rarely,frequently or no travel

In [9]:
data['Business_Travel_Rarely']=0
data['Business_Travel_Frequently']=0
data['Business_Non-Travel']=0

data.loc[data['BusinessTravel']=='Travel_Rarely','Business_Travel_Rarely'] = 1
data.loc[data['BusinessTravel']=='Travel_Frequently','Business_Travel_Frequently'] = 1
data.loc[data['BusinessTravel']=='Non-Travel','Business_Non-Travel'] = 1

## Categorising on the basis of education field

In [10]:
data['Life Sciences']=0
data['Medical']=0
data['Marketing']=0
data['Technical Degree']=0
data['Education Human Resources']=0
data['Education_Other']=0

data.loc[data['EducationField']=='Life Sciences','Life Sciences'] = 1
data.loc[data['EducationField']=='Medical','Medical'] = 1
data.loc[data['EducationField']=='Other','Education_Other'] = 1
data.loc[data['EducationField']=='Technical Degree','Technical Degree'] = 1
data.loc[data['EducationField']=='Human Resources','Education Human Resources'] = 1
data.loc[data['EducationField']=='Marketing','Marketing'] = 1

## Categorising on the basis of working department

In [11]:
data['Sales']=0
data['R&D']=0
data['Dept_Human Resources'] =0

data.loc[data['Department']=='Sales','Sales'] = 1
data.loc[data['Department']=='Research & Development','R&D'] = 1
data.loc[data['Department']=='Human Resources','Dept_Human Resources'] = 1

##  Setting predictor gender where male is indicated as 1 and female as 0

In [12]:
data.loc[data['Gender']=='Male','Gender'] = 1
data.loc[data['Gender']=='Female','Gender'] = 0

## Categorising on the basis of Job Role

In [13]:
data['Research Scientist']=0
data['Laboratory Technician']=0
data['Sales Executive']=0
data['Manufacturing Director']=0
data['Healthcare Representative']=0
data['Sales Representative']=0
data['Research Director']=0
data['Manager'] = 0
data['Job_Human_Resources'] = 0

data.loc[data['JobRole']=='Research Scientist','Research Scientist'] = 1
data.loc[data['JobRole']=='Laboratory Technician','Laboratory Technician'] = 1
data.loc[data['JobRole']=='Sales Executive','Sales Executive'] = 1
data.loc[data['JobRole']=='Sales Representative','Sales Representative'] = 1
data.loc[data['JobRole']=='Manufacturing Director','Manufacturing Director'] = 1
data.loc[data['JobRole']=='Healthcare Representative','Healthcare Representative'] = 1
data.loc[data['JobRole']=='Research Director','Research Director'] = 1
data.loc[data['JobRole']=='Manager','Manager'] = 1
data.loc[data['JobRole']=='Human Resources','Job_Human_Resources'] = 1
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,Dept_Human Resources,Research Scientist,Laboratory Technician,Sales Executive,Manufacturing Director,Healthcare Representative,Sales Representative,Research Director,Manager,Job_Human_Resources
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,0,...,0,0,0,1,0,0,0,0,0,0
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,1,...,0,1,0,0,0,0,0,0,0,0
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,4,1,...,0,0,1,0,0,0,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,0,...,0,1,0,0,0,0,0,0,0,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,1,...,0,0,1,0,0,0,0,0,0,0


## Categorising on the basis of Marital Satus of Employee

In [14]:
data['Marital_single']=0
data['Marital_married']=0
data['Marital_divorced']=0

data.loc[data['MaritalStatus']=='Married','Marital_married'] = 1
data.loc[data['MaritalStatus']=='Single','Marital_single'] = 1
data.loc[data['MaritalStatus']=='Divorced','Marital_divorced'] = 1

## Setting up the Over Time predictor

In [15]:
data.loc[data['OverTime']=='No','OverTime'] = 0
data.loc[data['OverTime']=='Yes','OverTime'] = 1
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,Sales Executive,Manufacturing Director,Healthcare Representative,Sales Representative,Research Director,Manager,Job_Human_Resources,Marital_single,Marital_married,Marital_divorced
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,0,...,1,0,0,0,0,0,0,1,0,0
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,1,...,0,0,0,0,0,0,0,0,1,0
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,4,1,...,0,0,0,0,0,0,0,1,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,0,...,0,0,0,0,0,0,0,0,1,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,1,...,0,0,0,0,0,0,0,0,1,0


## Checking for useless predictor variables and removing them

In [16]:
data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'Business_Travel_Rarely', 'Business_Travel_Frequently',
       'Business_Non-Travel', 'Life Sciences', 'Medical', 'Marketing',
       'Technical Degree', 'Education Human Resources', 'Education_Other',
       'Sales', 'R&D', 'Dept_Human Resources', 'Research Scientist',
       'Laboratory Technician', 'Sales Executive', 'Manufacturing Director',
       'Healthcare Represe

In [17]:
data = data.drop(['BusinessTravel','EducationField',
                        'Department','JobRole','MaritalStatus'],axis=1)
data.head()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,...,Sales Executive,Manufacturing Director,Healthcare Representative,Sales Representative,Research Director,Manager,Job_Human_Resources,Marital_single,Marital_married,Marital_divorced
0,41,1,1102,1,2,2,0,94,3,2,...,1,0,0,0,0,0,0,1,0,0
1,49,0,279,8,1,3,1,61,2,2,...,0,0,0,0,0,0,0,0,1,0
2,37,1,1373,2,2,4,1,92,2,1,...,0,0,0,0,0,0,0,1,0,0
3,33,0,1392,3,4,4,0,56,3,1,...,0,0,0,0,0,0,0,0,1,0
4,27,0,591,2,1,1,1,40,3,1,...,0,0,0,0,0,0,0,0,1,0


## Converting datatypes of some predictor variables

In [18]:
data.dtypes

Age                            int64
Attrition                     object
DailyRate                      int64
DistanceFromHome               int64
Education                      int64
EnvironmentSatisfaction        int64
Gender                        object
HourlyRate                     int64
JobInvolvement                 int64
JobLevel                       int64
JobSatisfaction                int64
MonthlyIncome                  int64
MonthlyRate                    int64
NumCompaniesWorked             int64
OverTime                      object
PercentSalaryHike              int64
PerformanceRating              int64
RelationshipSatisfaction       int64
StockOptionLevel               int64
TotalWorkingYears              int64
TrainingTimesLastYear          int64
WorkLifeBalance                int64
YearsAtCompany                 int64
YearsInCurrentRole             int64
YearsSinceLastPromotion        int64
YearsWithCurrManager           int64
Business_Travel_Rarely         int64
B

In [19]:
data['Attrition'] = data['Attrition'].astype('int')
data['Gender'] = data['Gender'].astype('int')
data['OverTime'] = data['OverTime'].astype('int')

## Finding coorelation among various predictors

In [20]:
data.corr()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,...,Sales Executive,Manufacturing Director,Healthcare Representative,Sales Representative,Research Director,Manager,Job_Human_Resources,Marital_single,Marital_married,Marital_divorced
Age,1.000000,-0.159205,0.010661,-0.001686,0.208034,0.010146,-0.036311,0.024287,0.029820,0.509604,...,-0.002001,0.049726,0.098825,-0.175785,0.185891,0.294248,-0.029856,-0.119185,0.083919,0.033120
Attrition,-0.159205,1.000000,-0.056652,0.077924,-0.031373,-0.103369,0.029453,-0.006846,-0.130016,-0.169105,...,0.019774,-0.082994,-0.078696,0.157234,-0.088870,-0.083316,0.036215,0.175419,-0.090984,-0.087716
DailyRate,0.010661,-0.056652,1.000000,-0.004985,-0.016806,0.018355,-0.011716,0.023381,0.046135,0.002966,...,-0.000513,-0.005302,0.040141,0.005375,-0.000021,-0.013224,-0.021156,-0.075835,0.040035,0.037080
DistanceFromHome,-0.001686,0.077924,-0.004985,1.000000,0.021042,-0.016075,-0.001851,0.031131,0.008783,0.005303,...,0.030761,0.011848,0.022916,-0.015994,-0.022351,-0.039190,-0.024089,-0.027445,0.030232,-0.005440
Education,0.208034,-0.031373,-0.016806,0.021042,1.000000,-0.027128,-0.016547,0.016775,0.042438,0.101589,...,0.053398,-0.005290,0.024270,-0.091465,0.049694,0.028453,-0.005295,0.004168,-0.001865,-0.002439
EnvironmentSatisfaction,0.010146,-0.103369,0.018355,-0.016075,-0.027128,1.000000,0.000508,-0.049857,-0.008278,0.001212,...,-0.024421,0.059178,0.014090,0.002949,-0.048689,0.010730,-0.022014,0.009035,-0.022180,0.016439
Gender,-0.036311,0.029453,-0.011716,-0.001851,-0.016547,0.000508,1.000000,-0.000478,0.017960,-0.039403,...,-0.005348,-0.065197,0.006823,-0.028877,-0.006121,-0.033880,0.036082,-0.032752,-0.007804,0.046076
HourlyRate,0.024287,-0.006846,0.023381,0.031131,0.016775,-0.049857,-0.000478,1.000000,0.042861,-0.027853,...,-0.011886,-0.014394,0.014599,-0.018703,-0.025128,0.012659,-0.016189,-0.033436,0.036432,-0.006150
JobInvolvement,0.029820,-0.130016,0.046135,0.008783,0.042438,-0.008278,0.017960,0.042861,1.000000,-0.012630,...,-0.011413,-0.021939,0.001272,-0.027282,0.015200,0.017112,-0.004952,-0.045253,0.028324,0.016815
JobLevel,0.509604,-0.169105,0.002966,0.005303,0.101589,0.001212,-0.039403,-0.027853,-0.012630,1.000000,...,0.127490,0.114896,0.115704,-0.216559,0.414319,0.552744,-0.100922,-0.087072,0.050547,0.037087


## Dividing data into train and test dataset

In [22]:
from sklearn.model_selection import train_test_split
#from random import seed

#seed(20)
train_x = data.drop(['Attrition'],axis=1)
train_y = data['Attrition']

X,test_x,Y,test_y = train_test_split(train_x, train_y, test_size=0.3,random_state=20)
len(test_x)

441

## Applying Linear Discriminant Analysis (LDA) to our data

In [23]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()
clf.fit(X,Y)

LinearDiscriminantAnalysis()

### Calculating accuracy of our model

In [24]:
from sklearn.metrics import accuracy_score

pred_y = clf.predict(test_x)

accuracy = accuracy_score(test_y, pred_y, normalize=True, sample_weight=None)
accuracy

0.8820861678004536

## Getting quantitative estimates of our model

In [25]:
from sklearn.metrics import classification_report

print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       371
           1       0.70      0.46      0.55        70

    accuracy                           0.88       441
   macro avg       0.80      0.71      0.74       441
weighted avg       0.87      0.88      0.87       441



#### We have applied linear discriminant analysis to the data getting an accuracy of 88.20% little bit higher then the logistic approach. As it is clear from our model that we are getting low value of recall for true value of attrition ,i.e., we are not getting enough of the relevant information of the attrited employees from the data. The retrieved model shows high senstivity but low specificity.

Now ,we will try a new model with few less relevant features trimmed out from our dataset.

## Applying Recursive Feature Elimination (RFE) for feature selection

In [27]:
from sklearn.feature_selection import RFE

rfe = RFE(clf)
rfe = rfe.fit(train_x,train_y)
print(rfe.support_)
print(rfe.ranking_)

[False False False False  True  True False  True False  True False False
 False  True False False False False False False  True False False False
 False False  True  True False False  True  True  True False False  True
  True  True  True False  True  True  True  True  True  True  True  True
  True]
[17 24 16 21  1  1 23  1 22  1 25 26 10  1 19 18  3  9 14  8  1 15 13 12
 11  4  1  1  7  6  1  1  1  5 20  1  1  1  1  2  1  1  1  1  1  1  1  1
  1]


### Transforming our data to desired no. of features

In [28]:
X =rfe.transform(X)
test_x = rfe.transform(test_x)
X.shape

(1029, 24)

### Calculating accuracy of our modified model

In [29]:
from sklearn.metrics import accuracy_score

clf.fit(X,Y)
pred_y = clf.predict(test_x)

accuracy = accuracy_score(test_y, pred_y, normalize=True, sample_weight=None)
accuracy

0.8684807256235828

### Quantitative estimates of our transformed model

In [30]:
from sklearn.metrics import classification_report

print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.89      0.96      0.92       371
           1       0.64      0.40      0.49        70

    accuracy                           0.87       441
   macro avg       0.77      0.68      0.71       441
weighted avg       0.85      0.87      0.86       441



#### Thus, we see a slight increase in accuracy of our model. We have trimmed our dataset to 40 features. This approach also shows considerable increase in precision,recall and F1 score .This ultimately results in increase of specificity of our model.This model also reduces our memory space and processing time as the operations to be performed are much less than former.

Trying with different number of features changes the accuracy of the model.

## Applying Quadratic Discriminant Analysis (QDA) to our data

In [31]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf = QuadraticDiscriminantAnalysis()
clf.fit(X,Y)

QuadraticDiscriminantAnalysis()

### Calculating accuracy of our model

In [32]:
from sklearn.metrics import accuracy_score

pred_y = clf.predict(test_x)

accuracy = accuracy_score(test_y, pred_y, normalize=True, sample_weight=None)
accuracy

0.3219954648526077

### Getting quantitative estimates of our model

In [33]:
from sklearn.metrics import classification_report

print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.91      0.22      0.35       371
           1       0.18      0.89      0.29        70

    accuracy                           0.32       441
   macro avg       0.54      0.55      0.32       441
weighted avg       0.79      0.32      0.34       441



It is clear that the **Quadratic Discriminant Analysis** is of no use to our data because of smaller dataset and larger variance.Thus, we go with previous two approaches of **Logistic Regression** and **Linear Discriminant Analysis** for our data.